# Network dimensionality and universality of rules

## For every considered model, vary (1) number and (2) width of layers. Otherwise same as 'comparing'.

Created by Basile Van Hoorick, Fall 2020.

In [3]:
%run FF_common.ipynb

In [ ]:
def get_metrics(hyperparams, all_stats):

    num_configs = len(hyperparams)
    test_accs_up = []
    test_accs_down = []
    speeds_down = []

    for i in range(num_configs):

        universal, l, w, cap = hyperparams[i]
    #     print('Universal:', universal)
    #     print('Number hidden layers:', l)
    #     print('Hidden layer width:', w)
    #     print('Cap:', cap)

        stats_up, stats_down = all_stats[i]
        stats_up = convert_multi_stats_uncertainty(stats_up)
        stats_down = convert_multi_stats_uncertainty(stats_down)
        test_acc_up = stats_up[2][-1]
        test_acc_down = stats_down[2][-1]
        speed_down = np.mean(stats_down[2]) / test_acc_down

    #     print(f'Final upstream test accuracy: {test_acc_up*100:.2f}%')
    #     print(f'Final downstream test accuracy: {test_acc_down*100:.2f}%')
    #     print(f'Downstream speed of convergence: {speed_down*100:.2f}%')
    #     print()

        test_accs_up.append(test_acc_up)
        test_accs_down.append(test_acc_down)
        speeds_down.append(speed_down)

    test_accs_up = np.array(test_accs_up).reshape((2, 3, 3))
    test_accs_down = np.array(test_accs_down).reshape((2, 3, 3))
    speeds_down = np.array(speeds_down).reshape((2, 3, 3))
    metrics = np.multiply(test_accs_up, test_accs_down, speeds_down)

    print('Upstream accuracy:')
    print(test_accs_up)
    print('Downstream accuracy:')
    print(test_accs_down)
    print('Downstream speed:')
    print(speeds_down)
    print('Product:')
    print(metrics)

    return metrics

In [7]:
# IMPORTANT: Henceforth, we use GD directly on inputs but use plasticity rules in the output and hidden layers.
opts = Options(gd_input=True,
               use_graph_rule=True,
               gd_graph_rule=True,
               use_output_rule=True,
               gd_output_rule=True,
               gd_output=False)
scheme = UpdateScheme(cross_entropy_loss=True,
                      mse_loss=False,
                      update_misclassified_only=False,
                      update_all_edges=True)

# Feed-forward brain config.
n_up = 16  # Input layer size for meta-learning.
n_down = 16  # Input layer size for desired task training.
n = 16  # Input layer size.
m = 2  # Output layer size.
p = 0.5  # Connectivity probability.

# Training config.
num_runs = 5
num_rule_epochs = 50
num_epochs_upstream = 1
num_epochs_downstream = 1
dataset = 'relu'

## PrePost

In [ ]:
hyperparams = []
all_stats = []

for universal in [False, True]:  # Specialized rules or not?
    
    for l in [2, 3, 4]:  # Number of hidden layers.
        
        for w in [16, 32, 64]:  # Width of hidden layers.
            
            cap = w // 2  # Number of nodes firing per layer.
            print('Universal:', universal)
            print('Number hidden layers:', l)
            print('Hidden layer width:', w)
            print('Cap:', cap)
            hyperparams.append((universal, l, w, cap))
            
            brain_prepost_fact = lambda: FFLocalNet(
                n, m, l, w, p, cap,
                hl_rules=TableRule_PrePost() if universal else [TableRule_PrePost()] * (l-1),
                output_rule=TableRule_PrePost(), options=opts, update_scheme=scheme)
            
            print('==== Interpretation: Pre and Post ====')
            cur_stats = evaluate_brain(
                brain_prepost_fact, n, dataset=dataset,
                num_runs=num_runs, num_rule_epochs=num_rule_epochs,
                num_epochs_upstream=num_epochs_upstream, num_epochs_downstream=num_epochs_downstream)
            all_stats.append(cur_stats)

../DataGenerator.py:34: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  out2 = F.softmax(torch.matmul(out1, w1))
  0%|          | 0/50 [00:00<?, ?it/s]

Universal: False
Number hidden layers: 2
Hidden layer width: 16
Cap: 8
==== Interpretation: Pre and Post ====

Run 1 / 5...
Count of 0: 3601  Count of 1: 2399
Meta-learning...


  3%|▎         | 48/1500 [00:00<00:03, 476.22it/s]

Last loss: 0.4477
Last train accuracy: 0.8513
Last test accuracy: 0.8507
Count of 0: 3868  Count of 1: 2132
Training SAME brain instance (NO backprop)...
===> WARNING: Backprop is disabled, which means that all layers without rules will never change their weights!
===> This is NOT recommended by Basile!
INITIAL train accuracy: 0.3540
INITIAL test accuracy: 0.3558
Epoch 1 / 1 ...


  0%|          | 0/50 [00:00<?, ?it/s]

...

  2%|▏         | 27/1500 [00:00<00:05, 269.96it/s]

Last loss: 0.3998
Last train accuracy: 0.8713
Last test accuracy: 0.8816
Count of 0: 4383  Count of 1: 1617
Training SAME brain instance (NO backprop)...
===> WARNING: Backprop is disabled, which means that all layers without rules will never change their weights!
===> This is NOT recommended by Basile!
INITIAL train accuracy: 0.7360
INITIAL test accuracy: 0.7287
Epoch 1 / 1 ...


  0%|          | 0/50 [00:00<?, ?it/s]


Last loss: 0.5591
Last train accuracy: 0.7360
Last test accuracy: 0.7287

Universal: False
Number hidden layers: 4
Hidden layer width: 64
Cap: 32
==== Interpretation: Pre and Post ====

Run 1 / 5...
Count of 0: 1057  Count of 1: 4943
Meta-learning...


 78%|███████▊  | 39/50 [02:52<00:48,  4.41s/it]

In [41]:
metrics_prepost = get_metrics(hyperparams_prepost, all_stats_prepost)

Upstream accuracy:
[[[0.7965 0.842  0.7067]
  [0.8167 0.6317 0.6444]
  [0.5897 0.8329 0.5356]]

 [[0.7553 0.8688 0.9225]
  [0.6796 0.7672 0.6177]
  [0.6493 0.6701 0.6564]]]
Downstream accuracy:
[[[0.6525 0.7045 0.6271]
  [0.7144 0.4112 0.654 ]
  [0.4228 0.7957 0.5444]]

 [[0.6033 0.7105 0.7515]
  [0.5811 0.7759 0.5424]
  [0.5217 0.4597 0.4817]]]
Downstream speed:
[[[0.5198 0.5932 0.4431]
  [0.5834 0.2598 0.4214]
  [0.2493 0.6628 0.2916]]

 [[0.4557 0.6173 0.6933]
  [0.3949 0.5952 0.3351]
  [0.3388 0.3081 0.3162]]]
Product:
[[[0.5198 0.5932 0.4431]
  [0.5834 0.2598 0.4214]
  [0.2493 0.6628 0.2916]]

 [[0.4557 0.6173 0.6933]
  [0.3949 0.5952 0.3351]
  [0.3388 0.3081 0.3162]]]


Three best configs:

* Universal, 2 layers, 64 width (metric 0.69)

* Non-universal, 4 layers, 32 width (metric 0.66)

* Universal, 2 layers, 32 width (metric 0.62)

## PrePostCount

In [ ]:
hyperparams_prepostcount = []
all_stats_prepostcount = []

for universal in [False, True]:  # Specialized rules or not?
    
    for l in [2, 3, 4]:  # Number of hidden layers.
        
        for w in [16, 32, 64]:  # Width of hidden layers.
            
            cap = w // 2  # Number of nodes firing per layer.
            print('Universal:', universal)
            print('Number hidden layers:', l)
            print('Hidden layer width:', w)
            print('Cap:', cap)
            hyperparams_prepostcount.append((universal, l, w, cap))
            
            brain_prepostcount_fact = lambda: FFLocalNet(
                n, m, l, w, p, cap,
                hl_rules=TableRule_PrePostCount() if universal else [TableRule_PrePostCount()] * (l-1),
                output_rule=TableRule_PrePostCount(), options=opts, update_scheme=scheme)
            
            print('==== Interpretation: PrePostCount ====')
            cur_stats = evaluate_brain(
                brain_prepostcount_fact, n, dataset=dataset,
                num_runs=num_runs, num_rule_epochs=num_rule_epochs,
                num_epochs_upstream=num_epochs_upstream, num_epochs_downstream=num_epochs_downstream)
            all_stats_prepostcount.append(cur_stats)

  0%|          | 0/50 [00:00<?, ?it/s]

Universal: False
Number hidden layers: 2
Hidden layer width: 16
Cap: 8
==== Interpretation: PrePostCount ====

Run 1 / 5...
Count of 0: 150  Count of 1: 5850
Meta-learning...


  3%|▎         | 45/1500 [00:00<00:03, 444.57it/s]

Last loss: 0.3387
Last train accuracy: 0.9767
Last test accuracy: 0.9744
Count of 0: 2779  Count of 1: 3221
Training SAME brain instance (NO backprop)...
===> WARNING: Backprop is disabled, which means that all layers without rules will never change their weights!
===> This is NOT recommended by Basile!
INITIAL train accuracy: 0.5273
INITIAL test accuracy: 0.5400
Epoch 1 / 1 ...


  0%|          | 0/50 [00:00<?, ?it/s]

...

  2%|▏         | 26/1500 [00:00<00:05, 249.73it/s]

INITIAL test accuracy: 0.3622
Epoch 1 / 1 ...


  0%|          | 0/50 [00:00<?, ?it/s]


Last loss: 0.7174
Last train accuracy: 0.6053
Last test accuracy: 0.6398

Universal: False
Number hidden layers: 4
Hidden layer width: 32
Cap: 16
==== Interpretation: PrePostCount ====

Run 1 / 5...
Count of 0: 4987  Count of 1: 1013
Meta-learning...


 60%|██████    | 30/50 [02:47<01:45,  5.29s/it]

In [ ]:
metrics_prepostcount = get_metrics(hyperparams_prepostcount, all_stats_prepostcount)

## PrePostPercent

In [ ]:
hyperparams_prepostpercent = []
all_stats_prepostpercent = []

for universal in [False, True]:  # Specialized rules or not?
    
    for l in [2, 3, 4]:  # Number of hidden layers.
        
        for w in [16, 32, 64]:  # Width of hidden layers.
            
            cap = w // 2  # Number of nodes firing per layer.
            print('Universal:', universal)
            print('Number hidden layers:', l)
            print('Hidden layer width:', w)
            print('Cap:', cap)
            hyperparams_prepostpercent.append((universal, l, w, cap))
            
            brain_prepostpercent_fact = lambda: FFLocalNet(
                n, m, l, w, p, cap,
                hl_rules=TableRule_PrePostPercent() if universal else [TableRule_PrePostPercent()] * (l-1),
                output_rule=TableRule_PrePostPercent(), options=opts, update_scheme=scheme)
            
            print('==== Interpretation: PrePostPercent ====')
            cur_stats = evaluate_brain(
                brain_prepostpercent_fact, n, dataset=dataset,
                num_runs=num_runs, num_rule_epochs=num_rule_epochs,
                num_epochs_upstream=num_epochs_upstream, num_epochs_downstream=num_epochs_downstream)
            all_stats_prepostpercent.append(cur_stats)

In [ ]:
metrics_prepostpercent = get_metrics(hyperparams_prepostpercent, all_stats_prepostpercent)